In [33]:
import pandas as pd
import sys
sys.path.insert(0, '../')
import constants as const
import src.utils as util
import src.calculate_errors as err

In [5]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [3]:
lstm_iter_results = pd.read_csv('../lstm_new_results/rmse.csv')

In [4]:
lstm_iter_results.head()

,Unnamed: 0,Grid,TL - Aggregated,SUB - Aggregated,PC - Aggregated,Site - Aggregated
0,32_3_0.001_1,0.461,0.455,0.447,0.342,0.342
1,32_3_0.001_3,0.440,0.448,0.491,0.357,0.355
2,32_3_0.001_7,0.465,0.446,0.472,0.373,0.363
3,32_3_0.0001_1,0.484,0.479,0.465,0.416,0.376
4,32_3_0.0001_3,0.451,0.441,0.454,0.452,0.485


In [19]:
fig = go.Figure()

for col in lstm_iter_results.columns[1:]:
    fig.add_trace(go.Scatter(x= lstm_iter_results['Unnamed: 0'], y=lstm_iter_results[col],  mode='markers', name=col))

In [20]:
fig.show()

In [18]:
lstm_iter_results.min()

Unnamed: 0           32_3_0.0001_1
Grid                          0.44
TL - Aggregated              0.441
SUB - Aggregated             0.441
PC - Aggregated              0.342
Site - Aggregated            0.332
dtype: object

In [21]:
cnn_iter_results = pd.read_csv('../cnn_new_results/rmse.csv')

In [23]:
cnn_iter_results.head()

,Unnamed: 0,Grid,TL - Aggregated,SUB - Aggregated,PC - Aggregated,Site - Aggregated
0,32_4_0.001_1,0.546,0.465,0.467,0.323,0.332
1,32_4_0.001_3,0.583,0.471,0.486,0.498,0.573
2,32_4_0.001_7,0.564,0.572,0.503,0.769,0.778
3,32_4_0.0001_1,0.456,0.405,0.430,0.331,0.328
4,32_4_0.0001_3,0.446,0.442,0.455,0.357,0.376


In [24]:
cnn_iter_results.min()

Unnamed: 0           32_4_0.0001_1
Grid                         0.427
TL - Aggregated              0.405
SUB - Aggregated             0.394
PC - Aggregated              0.323
Site - Aggregated            0.325
dtype: object

In [25]:
fig = go.Figure()

for col in cnn_iter_results.columns[1:]:
    fig.add_trace(go.Scatter(x= cnn_iter_results['Unnamed: 0'], y=cnn_iter_results[col],  mode='markers', name=col))
    
fig.show()

## Grid + PC level NN

In [40]:
def sum_fc_results(ts_array, model_path):
    dfs = []
    for ts in ts_array:
        ts_fc = pd.read_csv(f'../{model_path}/{ts}.csv', index_col=[0])[['fc']]
        dfs.append(ts_fc)
    concat_df = pd.concat(dfs, axis=1).sum(axis=1)
    return concat_df

In [41]:
def calculate_grid_error(start, end, model_path, error_metric):
    if end == -1:
        ts_array = const.TS[start: ]
    else:
        ts_array = const.TS[start: end]
    data = pd.read_csv(f'../ts_data/grid.csv', index_col=[0])
    look_back = 14 * 7  # 14 hours in to 7 days

    # train, val, test split
    train, val, test = util.split_hourly_data(data, look_back)
    train_df = train[['power']]
    denom = err.calculate_denom(train_df, const.H_SEASONALITY)
    
    results_df = pd.read_csv(f'../{model_path}/grid.csv', index_col=[0])
    test_sample = results_df['power'].values
    forecasts = sum_fc_results(ts_array, model_path).values
    horizon = 14
    
    if error_metric == "RMSE":
        mean_err, error_dist = err.test_errors_nrmse(train_df.values, test_sample, forecasts, horizon)
    else:
        mean_err, error_dist = err.test_errors(train_df, test_sample, forecasts, horizon, const.H_SEASONALITY, denom)
        
    return mean_err, error_dist

In [43]:
rmse_error, rmse_dist = calculate_grid_error(0, 1, 'combined_nn_results', "RMSE")

In [44]:
rmse_error

0.5857013669439975

In [45]:
def read_loss_curve():
    return pd.read_pickle(f'../combined_nn_results/training_loss_grid_iteration')

In [46]:
def plot_loss(loss_array, models):

    fig = go.Figure()
    
    i =0
    for loss in loss_array:
        fig.add_trace(go.Scatter(y= loss['loss'] , name = f'{models[i]}_train-loss' ))
        fig.add_trace(go.Scatter(y= loss['val_loss'], name = f'{models[i]}_val-loss'))
        i +=1
    fig.show()

In [47]:
loss = read_loss_curve()

In [48]:
plot_loss([loss], ['combined_nn'])